# CHOWDER

## Imports

In [ ]:
!pip install torchmetrics

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, ConcatDataset
import torch.nn as nn
import torch.optim as optim
import torchmetrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load Data

In [ ]:
processed_data_path = '/content/drive/My Drive/Breast_Cancer_Detection/Processed_Data/'

X_dev = np.load(processed_data_path + 'X_dev.npy')
y_dev = np.load(processed_data_path + 'y_dev.npy')

X_test = np.load(processed_data_path + 'X_test.npy')

## Split Data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.15, stratify=y_dev, random_state=42)

## Standardize Data

In [ ]:
def X_standardize(X_train, X_val, X_test):

    feature_mean = np.mean(X_train)
    feature_std = np.std(X_train)

    X_train_scaled = (X_train - feature_mean) / feature_std
    X_val_scaled = (X_val - feature_mean) / feature_std
    X_test_scaled = (X_test - feature_mean) / feature_std

    return X_train_scaled, X_val_scaled, X_test_scaled

In [ ]:
X_train_scaled, X_val_scaled, X_test_scaled = X_standardize(X_train, X_val, X_test)

## Convert to Tensor

In [ ]:
X_train_tensor = torch.Tensor(X_train_scaled)
X_val_tensor = torch.Tensor(X_val_scaled)
X_test_tensor = torch.Tensor(X_test_scaled)

In [ ]:
# Delete redundant variables to free up memory
del X_train_scaled
del X_val_scaled
del X_test_scaled
del X_train
del X_val
del X_test
del X_dev

## CHOWDER Model

In [ ]:
class CHOWDER(nn.Module):

    def __init__(self):
        super(CHOWDER, self).__init__()

        # Convolutional layer
        self.conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=2048, stride=2048)

        # MLP layers with dropout
        self.fc1 = nn.Sequential(
            nn.Linear(4, 200),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(200, 100),
            nn.Dropout(0.5)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(100, 1),
            nn.Dropout(0.5)
        )

    def forward(self, x):

        ## CONVOLUTION LAYER
        conv_output = self.conv(x)

        # Calculate L2-norm on weights in the convolutional layer
        l2_reg = 0.0
        for param in self.conv.parameters():
            l2_reg += torch.sum(param ** 2)

        ## MINMAX LAYER

        # Sort each row of the conv layer (each row is a sample)
        sorted_output, _ = torch.sort(conv_output, dim=2)

        # Number of top instances and negative evidence
        R = 2

        # Select the first two and last two sorted outputs
        selected_output = sorted_output[:, :, :R]
        a0 = torch.cat((selected_output, sorted_output[:, :, -R:]), dim=2)

        a0 = a0.squeeze()

        ## MLP

        # Layer 1
        z1 = self.fc1(a0)
        a1 = torch.sigmoid(z1)

        # Layer 2
        z2 = self.fc2(a1)
        a2 = torch.sigmoid(z2)

        # Output layer
        z3 = self.fc3(a2)
        output = torch.sigmoid(z3)

        return output, l2_reg

In [ ]:
demo_model = CHOWDER()

demo_model

CHOWDER(
  (conv): Conv1d(1, 1, kernel_size=(2048,), stride=(2048,))
  (fc1): Sequential(
    (0): Linear(in_features=4, out_features=200, bias=True)
    (1): Dropout(p=0.5, inplace=False)
  )
  (fc2): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): Dropout(p=0.5, inplace=False)
  )
  (fc3): Sequential(
    (0): Linear(in_features=100, out_features=1, bias=True)
    (1): Dropout(p=0.5, inplace=False)
  )
)

## Setup Hyperparameters and Data Loader

In [ ]:
# Define hyperparameters
BATCH_SIZE = 10
LR = 0.001
EPOCHS = 30
NUM_ENSEMBLE_MODELS = 10

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, data_tensor, target_tensor):
        self.data = data_tensor
        self.target = target_tensor

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y

    def __len__(self):
        return len(self.data)

# Create custom datasets
train_dataset = CustomDataset(X_train_tensor, torch.Tensor(y_train))
val_dataset = CustomDataset(X_val_tensor, torch.Tensor(y_val))

# Create DataLoader
train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_dl = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
# Free Up Memory
del X_train_tensor
del X_val_tensor
del y_train
del y_val

## Train Model and Validate

In [ ]:
# Define loss function and optimizer
loss_function = nn.BCELoss()

# Create an ensemble of models
ensemble_models = []

for _ in range(NUM_ENSEMBLE_MODELS):

    # Initialize the model
    model = CHOWDER()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=LR)
    ensemble_models.append((model, optimizer))

# Setup model counter
model_counter = 1

# Training and validation loops for each model in the ensemble
for model, optimizer in ensemble_models:

    print(f'\n ----- Model {model_counter} -----')
    model_counter += 1

    # Loop thorugh each epoch
    for epoch in range(EPOCHS):

        ## Training loop

        # Put model in train mode
        model.train()

        # Initialize loss, AUC and count
        total_loss = 0.0
        auroc_hist_train = 0.0
        total_count = 0.0

        # Train in batches
        for batch_x, batch_y in train_dl:

            batch_x_transformed = batch_x.unsqueeze(1)

            # Zero gradients
            optimizer.zero_grad()

            # Make predictions and get L2-norm from conv layer
            pred, weight_decay = model(batch_x_transformed)

            # Calculate loss
            batch_y = batch_y.view(-1, 1) # Reshape batch_y from (10) to (10,1)
            loss = loss_function(pred, batch_y) + (0.5*weight_decay)

            # Calculate gradients
            loss.backward()

            # Make step in gradient descent
            optimizer.step()

            # Add to loss counter for this epoch
            total_loss += loss.item() * len(batch_y)
            total_count += len(batch_y)

            # Calculate AUC for batch
            auroc_hist_train += torchmetrics.AUROC(task="binary")(pred, batch_y).item() * len(batch_y)

        # Calculate loss and AUC per sample
        train_average_loss = total_loss / total_count
        train_average_auc = auroc_hist_train/ total_count


        ## Validation loop

        # Put model in evaluation mode
        model.eval()

        # Fix gradients (only using model to predict)
        with torch.no_grad():

            # Initialize loss, AUC and count
            total_loss = 0.0
            auroc_hist_val = 0.0
            total_count = 0.0

            # Validate in batches
            for batch_x, batch_y in val_dl:

                batch_x_transformed = batch_x.unsqueeze(1)

                # Make predictions
                val_pred, _ = model(batch_x_transformed)

                # Calculate loss
                batch_y = batch_y.view(-1, 1) # Reshape batch_y from (10) to (10,1)
                loss = loss_function(val_pred, batch_y)

                # Add to loss for this epoch
                total_loss += loss.item() * len(batch_y)
                total_count += len(batch_y)

                # Calculate AUC for batch
                auroc_hist_val += torchmetrics.AUROC(task="binary")(val_pred, batch_y).item() * len(batch_y)

        # Calculate loss and AUC per sample
        val_average_loss = total_loss / total_count
        val_average_auc =  auroc_hist_val / total_count

        # Print results from each epoch
        print(f"Epoch [{epoch+1}/{EPOCHS}] - Train Loss: {train_average_loss:.4f} Train AUC Score: {train_average_auc:.4f} \
        Val Loss: {val_average_loss:.4f} Val AUC Score: {val_average_auc:.4f}")


 ----- Model 1 -----
Epoch [1/30] - Train Loss: 0.7366 Train AUC Score: 0.4549         Val Loss: 0.6863 Val AUC Score: 0.6748


/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch [2/30] - Train Loss: 0.6866 Train AUC Score: 0.4751         Val Loss: 0.6735 Val AUC Score: 0.5193
Epoch [3/30] - Train Loss: 0.6947 Train AUC Score: 0.4312         Val Loss: 0.6909 Val AUC Score: 0.4982
Epoch [4/30] - Train Loss: 0.7097 Train AUC Score: 0.4484         Val Loss: 0.6636 Val AUC Score: 0.5424
Epoch [5/30] - Train Loss: 0.6703 Train AUC Score: 0.5443         Val Loss: 0.6730 Val AUC Score: 0.6011
Epoch [6/30] - Train Loss: 0.6631 Train AUC Score: 0.5460         Val Loss: 0.6734 Val AUC Score: 0.3965
Epoch [7/30] - Train Loss: 0.6890 Train AUC Score: 0.4377         Val Loss: 0.6686 Val AUC Score: 0.6142
Epoch [8/30] - Train Loss: 0.6717 Train AUC Score: 0.5049         Val Loss: 0.6569 Val AUC Score: 0.6920
Epoch [9/30] - Train Loss: 0.7019 Train AUC Score: 0.4390         Val Loss: 0.6751 Val AUC Score: 0.5486
Epoch [10/30] - Train Loss: 0.6976 Train AUC Score: 0.3924         Val Loss: 0.6713 Val AUC Score: 0.4420
Epoch [11/30] - Train Loss: 0.6682 Train AUC Score: 0.

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch [24/30] - Train Loss: 0.6964 Train AUC Score: 0.4552         Val Loss: 0.6801 Val AUC Score: 0.4799
Epoch [25/30] - Train Loss: 0.6883 Train AUC Score: 0.4669         Val Loss: 0.6702 Val AUC Score: 0.4015
Epoch [26/30] - Train Loss: 0.6928 Train AUC Score: 0.4764         Val Loss: 0.6804 Val AUC Score: 0.3193
Epoch [27/30] - Train Loss: 0.6854 Train AUC Score: 0.4905         Val Loss: 0.6752 Val AUC Score: 0.4176
Epoch [28/30] - Train Loss: 0.6757 Train AUC Score: 0.5160         Val Loss: 0.6485 Val AUC Score: 0.7545
Epoch [29/30] - Train Loss: 0.6617 Train AUC Score: 0.5900         Val Loss: 0.6396 Val AUC Score: 0.5094
Epoch [30/30] - Train Loss: 0.6837 Train AUC Score: 0.4428         Val Loss: 0.6754 Val AUC Score: 0.4414

 ----- Model 6 -----
Epoch [1/30] - Train Loss: 0.7276 Train AUC Score: 0.5499         Val Loss: 0.6638 Val AUC Score: 0.4321
Epoch [2/30] - Train Loss: 0.6812 Train AUC Score: 0.4634         Val Loss: 0.6578 Val AUC Score: 0.4836
Epoch [3/30] - Train Loss:

In [ ]:
# After training all models, calculate the ensemble AUC

# Store predictions and labels
ensemble_predictions = []
val_labels = []

# Do not change gradients
with torch.no_grad():

    # Loop through each model
    for model, _ in ensemble_models:

        # Put model in evaluation mode
        model.eval()

        # Get predictions on the validation dataset
        predictions = []
        batch_labels = []  # Create a list to store labels for each batch

        for batch_x, batch_y in val_dl:

            batch_x_transformed = batch_x.unsqueeze(1)

            # Make predictions
            pred, _, _ = model(batch_x_transformed)
            # Save predictions
            predictions.append(pred)

            # Store val labels for this batch
            batch_labels.append(batch_y)

        # Concatenate the labels for this model
        batch_labels = torch.cat(batch_labels, dim=0)
        val_labels.append(batch_labels)

        predictions = torch.cat(predictions)
        ensemble_predictions.append(predictions)

# Concatenate all the validation labels
val_labels = torch.cat(val_labels, dim=0)

# Average the predictions from all models
ensemble_predictions = torch.stack(ensemble_predictions)
average_predictions = torch.mean(ensemble_predictions, dim=0)

# Calculate the AUC score based on the averaged predictions
average_auc = torchmetrics.AUROC(task="binary")(average_predictions, val_labels).item()
print(f"Ensemble AUC Score: {average_auc:.4f}")


## Test Model

In [ ]:
## Setup DataLoaders

# Create a dummy y for test set
y_test_dummy = torch.zeros(len(X_test_tensor),)

# Create custom datasets
test_dataset = CustomDataset(X_test_tensor, y_test_dummy)

# Create DataLoader
test_dl = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Merge them using ConcatDataset
dev_dataset = ConcatDataset([train_dataset, val_dataset])

# Create a DataLoader for the merged dataset
dev_dl = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
# Free Up Memory
del X_test_tensor
del train_dataset
del val_dataset

In [ ]:
# Define loss function and optimizer
loss_function = nn.BCELoss()

# Create an ensemble of models
ensemble_models = []

for _ in range(NUM_ENSEMBLE_MODELS):

    # Initialize the model
    model = CHOWDER()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=LR)
    ensemble_models.append((model, optimizer))

# Setup model counter
model_counter = 1

# Training loops for each model in the ensemble
for model, optimizer in ensemble_models:

    print(f'\n ----- Model {model_counter} -----')
    model_counter += 1

    # Loop thorugh each epoch
    for epoch in range(EPOCHS):

        ## Training loop

        # Put model in train mode
        model.train()

        # Initialize loss, AUC and count
        total_loss = 0.0
        auroc_hist_train = 0.0
        total_count = 0.0

        # Train in batches
        for batch_x, batch_y in dev_dl:

            # Add a dimension for channel numbers
            batch_x_transformed = batch_x.unsqueeze(1)

            # Zero gradients
            optimizer.zero_grad()

            # Add a dimension for channel numbers
            batch_x_transformed = batch_x.unsqueeze(1)

            # Make predictions and get L2-norm from conv layer
            pred, L2_term = model(batch_x_transformed)

            # Calculate loss
            batch_y = batch_y.view(-1, 1) # Reshape batch_y from (10) to (10,1)
            loss = loss_function(pred, batch_y) + (0.5*L2_term)

            # Calculate gradients
            loss.backward()

            # Make step in gradient descent
            optimizer.step()

            # Add to loss counter for this epoch
            total_loss += loss.item() * len(batch_y)
            total_count += len(batch_y)

            # Calculate AUC for batch
            auroc_hist_train += torchmetrics.AUROC(task="binary")(pred, batch_y).item() * len(batch_y)

        # Calculate loss and AUC per sample
        train_average_loss = total_loss / total_count
        train_average_auc = auroc_hist_train/ total_count

        # Print results from each epoch
        print(f"Epoch [{epoch+1}/{EPOCHS}] - Train Loss: {train_average_loss:.4f} Train AUC Score: {train_average_auc:.4f}")

In [ ]:
# After training all models, calculate the ensemble AUC

# Store predictions and labels
ensemble_predictions = np.zeros((len(test_dataset), len(ensemble_models)))

# Do not change gradients
with torch.no_grad():

    model_counter = 0

    # Loop through each model
    for model, _ in ensemble_models:

        # Put model in evaluation mode
        model.eval()

        # Get predictions on the validation dataset
        predictions = np.empty((0, 1))

        for batch_x, _ in test_dl:

            extra = 10 - len(batch_x)

            # If batch size is smaller than 10, pad rows in batch_x with 0s
            if extra > 0:
                pad_tensor = torch.zeros((extra,) + batch_x.shape[1:], dtype=batch_x.dtype)
                batch_x = torch.cat((batch_x, pad_tensor), dim=0)

            # Add a dimension for channel numbers
            batch_x_transformed = batch_x.unsqueeze(1)

            # Make predictions
            pred, _ = model(batch_x_transformed)

            # Save predictions
            pred_numpy = pred.numpy()
            predictions = np.concatenate((predictions, pred_numpy), axis=0)

            if extra > 0:
              predictions = predictions[:-extra]

        ensemble_predictions[:, model_counter] = predictions.squeeze()
        model_counter += 1

# Average the predictions from all models
average_prediction = np.mean(ensemble_predictions, axis=1)

In [ ]:
# Load metadata about each sample
data_path = '/content/drive/My Drive/Breast_Cancer_Detection/Data/'
df_test = pd.read_csv(data_path + "test_metadata.csv")

# Join sample ID metadata with probability prediction
CHOWDER_submission = pd.DataFrame( {"Sample ID": df_test["Sample ID"].values, "Target": average_prediction}).sort_values("Sample ID")

In [ ]:
def sanity_checks(submission):
    assert all(submission["Target"].between(0, 1)), "`Target` values must be in [0, 1]"
    assert submission.shape == (149, 2), "Your submission file must be of shape (149, 2)"
    assert list(submission.columns) == ["Sample ID", "Target",], "Your submission file must have columns `Sample ID` and `Target`"

sanity_checks(CHOWDER_submission)

In [ ]:
submission_path = '/content/drive/My Drive/Breast_Cancer_Detection/Predictions/'

CHOWDER_submission.to_csv(submission_path + "CHOWDER_submission_base.csv", index=None)